In [3]:
# import libraries
import pandas as pd
import numpy as np
from json import loads
from ast import literal_eval
import glob
import random
import networkx as nx
import osmnx as ox
from pathlib import Path
from sklearn.neighbors import BallTree
from IPython.display import clear_output
import timeit
from numba import jit
from dask import dataframe as dd
from dask.multiprocessing import get
from multiprocessing import cpu_count
from dask.diagnostics import ProgressBar

In [2]:
# use all cores from desktop when performing parallel processing
nCores = cpu_count()

## Helper Functions

In [6]:
# delete variable, if it exists
def deleteIfExists(df):
    try:
        df
    except NameError:
        return False
    else:
        del df
        return True # successfully deleted variable

## Correct malformed json in Jams dataset

In [ ]:
# set path to input file
filepath = 'data/jams-head.csv'

In [ ]:
# create valid dataset
with open(f'{filepath}') as in_file:
    with open(f'{filepath[:-4]}-valid.csv', 'w') as out_file:
        for line in in_file:
            out_line = []
            
            is_list = 0
            in_quotes = False
            is_json = False # flag to track if reading JSON body

            for c in line: # c = char
                # "[{'"x'": -118.342949, '"y'": 34.078279}, ...]"
                if c is '[':
                    out_line[-1] = "'" # replace first " to '
                    out_line.append(c)
                    is_json = True
                    continue
                elif c is ']':
                    out_line.append(c)
                    is_list = 1 # at closing bracket
                    is_json = False
                    continue
                    
                # preserve "Los Angeles, CA" so comma doesn't become a tab
                elif c is '"': 
                    if not in_quotes:
                        in_quotes = True
                    else:
                        in_quotes = False
                
                # change " to ' at end of list
                if is_list is 1:
                    out_line += "'"
                    is_list = 2 # done processing json
                    continue
                    
                # reconstruct line to write to  file
                if not is_json:
                    if c is not ',':
                        out_line.append(c)
                    else:
                        if not in_quotes:
                            out_line.append('\t')
                        else:
                            out_line.append(c)
                elif is_json and (c is not "\'"): # skip the double quote that creates invalid json
                    out_line.append(c)
                elif is_json and (c is '\''): # pass inconsistent pattern
                    pass
            
            # output each row to file
            out_file.write(''.join(out_line[:-1]) + '\n')


In [ ]:
# load valid data into dataframe
df = pd.DataFrame()
df = pd.read_csv(f'{filepath[:-4]}-valid.csv', sep='\t', header=None)
df.columns = ['primary_id1','primary_id2','pub_millis','pub_utc_date','unknown1','start_node','road_type','end_node','interm_city','country','delay_seconds','avg_speed','speed_kmh','length_meter','turn_type','level','blocking_alert_uuid','line','traffic_type','turn_line','unknown2','datafile_id']


In [ ]:
# isolate relevant data
df = df[['pub_millis', 'avg_speed', 'line']]

In [ ]:
# cast to datetime object
df['pub_millis'] = pd.to_datetime(df['pub_millis'], unit='ms')

## save state 1

In [3]:
df.to_csv('data/jams-refined.csv', sep='\t', index=False, header=False)

## Partition Jams into smaller datasets to process easier

In [9]:
deleteIfExists(df)

In [ ]:
df = pd.DataFrame()
df = pd.read_csv('data/jams-refined.csv', sep='\t', header=None)
df.columns = ['pub_millis', 'avg_speed', 'line']

In [ ]:
# get total number of rows in dataframe
df_length = df.count()

# create arbitrary partition size
n = df_length * 0.3

file_num = 1

# define starting and ending spliced index for dataframe
starting_index = 0
ending_index = n

# while the end index is not out of bounds
while ending_index <= df_length:
    # splice the dataframe and save the partitions
    df.iloc[starting_index:ending_index].to_csv(f'data/jams-partition-{file_num}.csv', sep='\t', index=False, header=False)
    
    file_num += 1
    starting_index = ending_index
    
    if ending_index + n > df_length:
        ending_index = df_length
    else:
        ending_index += n

## Process each Jams partition
These following steps will be repeated until all partitions complete save state 2.

In [7]:
file_num = 1
df = pd.DataFrame()
df = pd.read_csv(f'data/jams-partition-{file_num}.csv', sep='\t', header=None)

# pub_millis: dateobject, avg_speed: float, line: list of dict
df.columns = ['pub_millis', 'avg_speed', 'line']

In [8]:
df.head()

,pub_millis,avg_speed,line
0,2018-04-10 23:57:34.974,5.89444,"'[{""x"": -118.271445, ""y"": 34.062425}, {""x"": -1..."
1,2018-04-11 00:30:30.955,4.35000,"'[{""x"": -118.273876, ""y"": 34.072198}, {""x"": -1..."
2,2018-04-11 00:42:26.705,1.27778,"'[{""x"": -118.271838, ""y"": 34.06928}, {""x"": -11..."
3,2018-04-10 22:45:14.498,1.78889,"'[{""x"": -118.260378, ""y"": 34.078331}, {""x"": -1..."
4,2018-04-11 00:47:49.429,5.55556,"'[{""x"": -118.261324, ""y"": 34.076332}, {""x"": -1..."


## Map waze coordinates to edges from graph

In [9]:
# load road network of Los Angeles City
G = ox.graph_from_place('Los Angeles City, Los Angeles, CA',network_type='drive')
gdf, _ = ox.graph_to_gdfs(G)

In [11]:
# check specific edge from road network
G[402279372][123306201]

AtlasView({0: {'osmid': 13397064, 'name': 'Knox Street', 'highway': 'residential', 'oneway': False, 'length': 417.085}})

## Skip to Ball Tree if node.csv, segment.csv, selSegs_1.csv are already created

In [12]:
# node.csv contains all the intersections in LA
# header: osm_id, latitude, longitude
with open('data/node.csv', 'w') as out_file:
    for osm_id in G.node:
        node = G.node[osm_id]
        out_file.write(f"{osm_id},{node['y']},{node['x']}\n")

                       
# segment.csv contains all of the road segments in LA
# header: pseudo_edge_id, start_osmid, end_osmid
osm_edge_ids = nx.edges(G)
row_counter = 0
edge_dict = {}
                       
with open('data/segment.csv', 'w') as out_file:
    for x, y in osm_edge_ids:
        # prevent duplicated edges from getting added to the segments file
        if (x, y) not in edge_dict:
            out_file.write(f'{row_counter},{x},{y}\n')
            edge_dict[(x, y)] = row_counter
            row_counter += 1

del edge_dict

In [4]:
# load segment.csv data
segments = {}
with open('data/segment.csv') as in_file:
    for line in in_file:
        split_line = line.rstrip().split(',')
        
        # set (node tuple)-key in dict to its (rowid)-value
        segments[(int(split_line[1]), int(split_line[2]))] = int(split_line[0])

# v: pseudo-edge id
# k: start and end nodes
inverted_segments = {v: k for k, v in segments.items()}

In [14]:
# get list of edge ids that exist in the financial district
fin_g = ox.graph_from_place('Financial District, Los Angeles, CA',network_type='drive')
district_edge_ids = []

for x, y in nx.edges(fin_g):
    district_edge_ids.append(segments[(x, y)])
district_edge_ids.sort()

print(district_edge_ids)

[4214, 6273, 6274, 7997, 14074, 15945, 15969, 15971, 22813, 26311, 26312, 33369, 33370, 41472, 41496, 41497, 41498, 42385, 42386, 58386, 58387, 58388, 59425, 59426, 59447, 59448, 59449, 59456, 59457, 59495, 59496, 60896, 60898, 61634, 61635, 63069, 63235, 63236, 63245, 63246, 63247, 63248, 68997, 68998, 68999, 69733, 69735, 69736, 70702, 70703, 73873, 73874, 77168, 77169, 77174, 77176, 82431, 83352, 83353, 83812, 83813, 84020, 84021, 86817, 86819, 86820, 86822, 86823, 86824, 88491, 88492, 88493, 88827, 89808, 89809, 90175, 90176, 90177, 90178, 90442, 90443, 90444, 90537, 94602, 94603, 94604, 106473, 108638, 109253, 109255, 109256, 118097, 120242, 120243, 120247, 120286, 120287, 120555, 120556, 120563, 120564, 120572, 120573, 132535, 132536, 132543, 132544, 132545, 134117, 134118, 134177]


In [15]:
# selSegs_1.csv contains the road segments of interest for prediction
with open('data/selSegs_1.csv', 'w') as out_file:
    for edge_id in district_edge_ids:
        out_file.write(f'{edge_id}\n')

## Ball Tree. This section maps coordinates from Waze linestrings to the closest intersections

In [17]:
# initialize Ball Tree algorithm using osm road network
ball_tree = BallTree(gdf[['y', 'x']], metric='haversine')

In [16]:
def create_edges(nodes):
    edges = []
    
    for i in range(len(nodes) - 1):
        if (nodes[i], nodes[i+1]) in segments:
            edges.append((nodes[i], nodes[i+1]))    
            
    return edges

In [18]:
@jit #numba magic
def map_edges(line_string):
    closest_nodes = []
    
    # store coordinates in line
    line = loads(line_string.replace('\'', ''))
    
    # iterate over each coordinate in line to get closest nodes
    for coordinate in line:
        _, indexes = ball_tree.query([[coordinate['y'], coordinate['x']]])
        closest_nodes.append(int(gdf.iloc[indexes[0]].osmid.tolist()[0]))
    
    # get all unique closest nodes
    closest_nodes = list(set(closest_nodes))
    
    return create_edges(closest_nodes)

In [19]:
# parallelize using dask and numba
res = dd.from_pandas(df,npartitions=nCores).map_partitions(
    lambda _df : _df.apply(
        lambda x : map_edges(x.line),axis=1), meta=('edges','i8'))

# print progress of parallelization
with ProgressBar(dt=0.01):
    out = res.compute(scheduler='processes')

[########################################] | 100% Completed | 14hr 28min 46.8s


In [20]:
# print results from parallelization
out.tail()

21690695                                                  []
21690696    [(122532560, 122735795), (123129231, 122527731)]
21690697                                                  []
21690698                                                  []
21690699                                                  []
Name: edges, dtype: object

In [21]:
# store output from map_edges into the edges column
df['edges'] = pd.Series(out)

In [22]:
# delete linestring column now that we have the edges
del df['line']

In [23]:
df.tail()

,pub_millis,avg_speed,edges
21690695,2018-07-06 17:28:12.166,7.35278,[]
21690696,2018-07-06 17:28:12.867,1.77222,"[(122532560, 122735795), (123129231, 122527731)]"
21690697,2018-07-06 16:18:23.454,13.52780,[]
21690698,2018-07-06 17:23:19.059,19.95560,[]
21690699,2018-07-06 17:24:31.421,6.39167,[]


## save state 2

In [24]:
df.to_csv(f'data/jams-edges-{file_num}.csv', index=False, header=False)

## <span style='color:red;'>Note: Finish creating all jams-edges files to continue</span>

## Load series of jams-edges files to continue processing

In [ ]:
deleteIfExists(df)

In [7]:
# read in a series of jams-edge files using glob
df = pd.DataFrame()
df = pd.concat([pd.read_csv(f, header=None) for f in glob.glob('data/jams-edges-*.csv')], ignore_index=True)

# pub_millis: dateobject, avg_speed: float, line: list of dict
df.columns = ['pub_millis', 'avg_speed', 'edges']

In [8]:
df.head()

,pub_millis,avg_speed,edges
0,2018-04-10 23:57:34.974,5.89444,[]
1,2018-04-11 00:30:30.955,4.35000,"[(122663681, 122663684), (122663684, 122663687..."
2,2018-04-11 00:42:26.705,1.27778,"[(60946388, 122688732)]"
3,2018-04-10 22:45:14.498,1.78889,[]
4,2018-04-11 00:47:49.429,5.55556,"[(21063622, 122814825), (122814825, 60946379),..."


In [9]:
# remove rows with empty edge lists
df = df[df['edges'] != '[]'].reset_index(drop=True)

df.head()

,pub_millis,avg_speed,edges
0,2018-04-11 00:30:30.955,4.35000,"[(122663681, 122663684), (122663684, 122663687..."
1,2018-04-11 00:42:26.705,1.27778,"[(60946388, 122688732)]"
2,2018-04-11 00:47:49.429,5.55556,"[(21063622, 122814825), (122814825, 60946379),..."
3,2018-04-11 00:41:03.941,1.14722,"[(14940283, 122967892)]"
4,2018-04-10 23:27:53.283,1.32222,"[(122681252, 122681255)]"


## Create pseudo edge-id

In [15]:
# create dict to store key-value pairs of (start_node, end_node): edge_id
segments = {}
with open('data/segment.csv') as in_file:
    for line in in_file:
        split_line = line.rstrip().split(',')
        
        # set node tuple (key) in dict to its rowid (value)
        start = int(split_line[1])
        end = int(split_line[2])
        edge_id = int(split_line[0])
        segments[start, end] = edge_id

# v = edges_id, k = (start, end)
inverted_segments = {v: k for k, v in segments.items()}

In [17]:
def create_pseudo_edge_id(col):
    from ast import literal_eval
    edge_indexes = []
    for edge in literal_eval(col):
        # dict.get(item if exist, else default to value)
        edge_indexes.append(segments.get(edge, -1))
    return edge_indexes

In [18]:
# map each edge tuple to a pseudo edge id
res = dd.from_pandas(df,npartitions=nCores).map_partitions(
    lambda _df : _df.apply(
        lambda x : create_pseudo_edge_id(x.edges),axis=1), meta=('edges','i8'))

# print progress
with ProgressBar(dt=0.01):
    out = res.compute(scheduler='processes')

[########################################] | 100% Completed | 17min 40.1s


In [19]:
out.head()

0    [116529, 116533, 49632, 116509, 135195, 116518]
1                                           [135193]
2                               [97216, 275, 108197]
3                                           [135466]
4                                           [135124]
Name: edges, dtype: object

In [20]:
# assign the pseudo edge id lists to the dataframe
df['edge_id'] = pd.Series(out)

In [21]:
df.head()

,pub_millis,avg_speed,edges,edge_id
0,2018-04-11 00:30:30.955,4.350000,"[(122663681, 122663684), (122663684, 122663687...","[116529, 116533, 49632, 116509, 135195, 116518]"
1,2018-04-11 00:42:26.705,1.277780,"[(60946388, 122688732)]",[135193]
2,2018-04-11 00:47:49.429,5.555560,"[(21063622, 122814825), (122814825, 60946379),...","[97216, 275, 108197]"
3,2018-04-11 00:41:03.941,1.147220,"[(14940283, 122967892)]",[135466]
4,2018-04-10 23:27:53.283,1.322220,"[(122681252, 122681255)]",[135124]
5,2018-04-11 00:37:17.085,6.761110,"[(123368236, 122928109), (28112699, 123065661)]","[27371, 65706]"
6,2018-04-11 00:05:53.054,2.330560,"[(28112695, 122517974)]",[65693]
7,2018-04-11 00:30:31.632,4.386110,"[(1721407624, 1721407626)]",[39692]
8,2018-04-11 00:39:39.774,5.255560,"[(123368236, 122928109)]",[27371]
9,2018-04-11 00:08:14.911,3.136110,"[(123179440, 15864734), (28112697, 123179450)]","[108527, 65698]"


In [22]:
# drop edges column from dataframe and reset index
df = df.drop(['edges'], axis=1).reset_index(drop=True)

In [23]:
# show the results of the drop operation
df.head()

,pub_millis,avg_speed,edge_id
0,2018-04-11 00:30:30.955,4.35000,"[116529, 116533, 49632, 116509, 135195, 116518]"
1,2018-04-11 00:42:26.705,1.27778,[135193]
2,2018-04-11 00:47:49.429,5.55556,"[97216, 275, 108197]"
3,2018-04-11 00:41:03.941,1.14722,[135466]
4,2018-04-10 23:27:53.283,1.32222,[135124]


## save state 3

In [26]:
df.to_csv('data/jams-edgeid.csv', index=False, header=False)

# Reload jams-edgeid to continue processing

In [ ]:
deleteIfExists(df)

In [172]:
df = pd.DataFrame()
df = pd.read_csv('data/jams-edgeid.csv', header=None, names=['pub_millis', 'avg_speed', 'edge_id'])

In [173]:
%%time
from ast import literal_eval
df['edge_id'] = df['edge_id'].apply(literal_eval)

Wall time: 5min 26s


In [174]:
# splits the list of edge ids into multiple rows (one row per edge id)
a = 'pub_millis'
b = 'edge_id'
c = 'avg_speed'

speed_df = pd.DataFrame({'pub_millis': np.repeat(df[a].values, df[b].str.len()),
                        'avg_speed': np.repeat(df[c].values, df[b].str.len()),
                        'edge_id': np.concatenate(df[b].values)})

In [175]:
# show original state of df before splitting edge id into multiple rows
df.head()

del df

,pub_millis,avg_speed,edge_id
0,2018-04-11 00:30:30.955,4.35000,"[116529, 116533, 49632, 116509, 135195, 116518]"
1,2018-04-11 00:42:26.705,1.27778,[135193]
2,2018-04-11 00:47:49.429,5.55556,"[97216, 275, 108197]"
3,2018-04-11 00:41:03.941,1.14722,[135466]
4,2018-04-10 23:27:53.283,1.32222,[135124]


In [176]:
# show the result of splitting the edge ids into multiple rows
speed_df.head()

,pub_millis,avg_speed,edge_id
0,2018-04-11 00:30:30.955,4.35,116529
1,2018-04-11 00:30:30.955,4.35,116533
2,2018-04-11 00:30:30.955,4.35,49632
3,2018-04-11 00:30:30.955,4.35,116509
4,2018-04-11 00:30:30.955,4.35,135195


In [121]:
# define interval for time series
interval = 10
interval_string = str(interval) + 'min'

In [122]:
# round each time in the dataset to the nearest interval
speed_df['pub_millis'] = pd.to_datetime(speed_df['pub_millis']).dt.round(interval_string)

In [125]:
# groupby edge and time then find the mean of speeds 
# for rows with the same timestamp and edge id
speed_df = speed_df.groupby(['pub_millis', 'edge_id'], as_index=False)['avg_speed'].mean()
speed_df = speed_df.sort_values(['edge_id', 'pub_millis'], ascending=[True, True]).reset_index(drop=True)

In [126]:
# filter out all rows that have an edge_id of -1 because they weren't found
speed_df = speed_df[speed_df['edge_id'] != -1]

In [127]:
speed_df.head(10)

,pub_millis,edge_id,avg_speed
0,2017-12-18 04:30:00,2,1.811110
1,2017-12-25 06:30:00,2,1.974070
2,2018-02-24 01:30:00,2,2.516670
3,2018-03-06 04:30:00,2,2.227780
4,2018-03-22 15:30:00,2,2.561110
5,2018-05-23 01:00:00,2,1.600000
6,2018-06-25 09:00:00,2,1.636805
7,2017-12-15 02:00:00,5,2.444841
8,2018-01-07 07:30:00,5,2.791358
9,2018-01-21 10:00:00,5,2.645000


## save state 4

In [128]:
speed_df.to_csv(f'data/jams-speed-{interval}.csv', index=False, header=False)

## Reload jams-speed to continue processing

In [ ]:
deleteIfExists(speed_df)

In [130]:
df = pd.DataFrame()
df = pd.read_csv('data/jams-speed.csv', header=None)

# pub_millis: dateobject, avg_speed: float, line: list of dict
df.columns = ['pub_millis', 'edge_id', 'avg_speed']

In [131]:
# create segments dict using segment.csv
segments = {}
with open('data/segment.csv') as in_file:
    for line in in_file:
        split_line = line.rstrip().split(',')
        
        # set the key of dict to be the edge tuple, and the value to be the row id
        segments[(int(split_line[1]), int(split_line[2]))] = int(split_line[0])

In [132]:
df.head()

,pub_millis,edge_id,avg_speed
0,2017-12-18 04:30:00,2,1.81111
1,2017-12-25 06:30:00,2,1.97407
2,2018-02-24 01:30:00,2,2.51667
3,2018-03-06 04:30:00,2,2.22778
4,2018-03-22 15:30:00,2,2.56111


In [133]:
# filter edge-ids for financial district
financial_district_ids = []
with open('data/selSegs_1.csv') as in_file:
    for line in in_file:
        financial_district_ids.append(int(line.rstrip()))
        
financial_district_ids[0:5]

[4214, 6273, 6274, 7997, 14074]

In [134]:
# create dict of edge ids and tuple data for financial district
district_segments = {}
with open('data/segment.csv') as in_file:
    for line in in_file:
        split_line = line.rstrip().split(',')
        
        if int(split_line[0]) in financial_district_ids:
            # set the key of dict to be the edge tuple, and the value to be the row id
            district_segments[int(split_line[0])] = (int(split_line[1]), int(split_line[2]))

In [135]:
# filter dataframe by all edge ids that exist in the financial district
district_df = df[df['edge_id'].isin(financial_district_ids)]
district_df['edge_id'].unique()

array([  4214,   6274,  26311,  26312,  41496,  41497,  42385,  42386,
        58388,  59425,  59426,  59449,  59456,  59496,  60896,  60898,
        63235,  63236,  63245,  63246,  63247,  68997,  68998,  68999,
        69733,  69735,  77169,  77174,  77176,  83353,  83813,  88491,
        88492,  88493,  88827,  89809,  90176,  90177,  90178,  90442,
        90443,  90444,  94602, 118097, 120247, 120563, 120572, 132535,
       132544, 132545, 134118, 134177], dtype=int64)

In [136]:
district_df.size

137283

In [137]:
district_df.head()

,pub_millis,edge_id,avg_speed
362127,2017-12-12 02:30:00,4214,1.819445
362128,2017-12-13 00:00:00,4214,1.745554
362129,2017-12-13 00:30:00,4214,1.936668
362130,2017-12-13 01:00:00,4214,1.794841
362131,2017-12-13 01:30:00,4214,1.785187


## save state 5

In [138]:
district_df.to_csv(f'data/jams-district-{interval}.csv', index=False, header=False)

## Impute missing values using KNN Regessor

In [ ]:
deleteIfExists(district_df)

In [ ]:
# define interval for time series
interval = 10

In [ ]:
df = pd.DataFrame()
df = pd.read_csv(f'data/jams-district-{interval}.csv', dtype={'avg_speed':'float64'}, header=None,
                     names=['pub_millis', 'edge_id', 'avg_speed'],
                     converters={'edge_id':literal_eval, 'pub_millis':pd.to_datetime})

In [ ]:
# create a dict of edges
segments = {}
with open('data/segment.csv') as in_file:
    for line in in_file:
        split_line = line.rstrip().split(',')
        
        # set the key of dict to be the edge tuple, and the value to be the row id
        segments[(int(split_line[1]), int(split_line[2]))] = int(split_line[0])

# invert segments dict 
inverted_segments = {v: k for k, v in segments.items()}

## Format data into time series

In [ ]:
# reload road network
G = ox.graph_from_place('Los Angeles City, Los Angeles, CA',network_type='drive')
district_df = pd.DataFrame(columns=['pub_millis', 'edge_id', 'avg_speed'])

# convert pub_millis to datetime
df['pub_millis'] = pd.to_datetime(df['pub_millis'])

# find earliest time and use as starting pseudo-index
start_time = df['pub_millis'].min()

# total number of intervals = (60 minutes / (m minutes per interval)) * (24 hours/day) * (d days)
# currently generating 44 days worth of data
data_points = int((60 / int(interval)) * 24 * 44)
data = [{} for _ in range(data_points)]

# initialize previous_edge_id
previous_edge_id = df.iloc[0]['edge_id']

In [ ]:
# show starting time
start_time

In [ ]:
# find the ending time based on the total number of intervals for a segment
end_time = start_time + np.timedelta64(interval * (data_points - 1), 'm')
end_time

In [ ]:
# filter the data to only include data points before the end time
df = df[df['pub_millis'] < end_time]
df.head(10)

## Identify where there are missing data and fill with empty string

In [ ]:
for i, row in df.iterrows():

    # find matching edge_id
    if previous_edge_id == row['edge_id']:
        
        # index = (total_seconds / (60 seconds/min * interval)) % data_points
        delta_time = row['pub_millis'] - start_time
        
        data[int((delta_time.total_seconds() / (60.0 * interval)) % data_points)] = {
            'pub_millis': row['pub_millis'],
            'edge_id': row['edge_id'],
            'avg_speed': row['avg_speed']
        }
        
    else:  # fill in missing data with empty string
        for index in range(data_points):
            if data[index] == {}:
                data[index] = {
                    'pub_millis': start_time + np.timedelta64(interval * index, 'm'),
                    'edge_id': previous_edge_id,
                    'avg_speed': ''
                }

        # append edge data to dataframe
        district_df = district_df.append(pd.DataFrame.from_dict(data), sort=False).reset_index(drop=True)

        # update previous edge
        previous_edge_id = row['edge_id']

        # reset data because data list is only associated with one edge at a time
        data = [{} for _ in range(data_points)]
    

In [ ]:
# accounts for the last edge id from the dataframe 
for index in range(data_points):
    if data[index] == {}:
        data[index] = {
            'pub_millis': start_time + np.timedelta64(interval * index, 'm'),
            'edge_id': previous_edge_id,
            'avg_speed': ''
        }

# append edge data to dataframe
district_df = district_df.append(pd.DataFrame.from_dict(data), sort=False).reset_index(drop=True)

In [ ]:
# create list of pseudo-edge-ids for financial district
financial_district_ids = []
edge_count = 0

with open('data/selSegs_1.csv') as in_file:
    for line in in_file:
        financial_district_ids.append(int(line.rstrip()))
        edge_count += 1

print(edge_count)

In [ ]:
# display the first ten ids of financial district
financial_district_ids[0:10]

In [ ]:
# verify edge ids are in financial district
district_df = district_df[district_df['edge_id'].isin(financial_district_ids)]

In [ ]:
# if edge ids are accounted for, then remove from financial_district_ids
for district_id in district_df['edge_id'].unique().tolist():
    try: 
        financial_district_ids.remove(district_id)
    except: # ignore any ids that are not in financial district
        pass


In [ ]:
# display the edge ids that need to be computed
print(financial_district_ids)

In [ ]:
# create dict of edge ids with pairs of start and end nodes for financial district
district_segments = {}
with open('data/segment.csv') as in_file:
    for line in in_file:
        split_line = line.rstrip().split(',')
        
        if int(split_line[0]) in financial_district_ids:
        
            # set the key of dict to be the row id, and the value to be the start and end node pair
            district_segments[int(split_line[0])] = (int(split_line[1]), int(split_line[2]))

In [ ]:
# filling in missing data with empty string
data = [{} for _ in range(data_points)]

for district_id in financial_district_ids:
    for index in range(data_points):
        if data[index] == {}:
            data[index] = {
                'pub_millis': start_time + np.timedelta64(interval * index, 'm'),
                'edge_id': district_id,
                'avg_speed': ''
            }

    # append edge data to dataframe
    district_df = district_df.append(pd.DataFrame.from_dict(data), sort=False).reset_index(drop=True)

    # update previous edge
    data = [{} for _ in range(data_points)]

In [ ]:
# Time of Day labels:
# 4 <= em < 6 (early morning)
# 6 <= mr < 9 (morning rush)
# 9 <= m < 12 (morning)
# 12 <= a < 15 (afternoon)
# 15 <= ar < 19 (afternoon rush)
# 19 <= n < 22 (night)
# 22 <= nn < 4 (night night)

time_of_day_labels = []
hour_of_day = -1

day_of_week_labels = []
day_of_week = -1

season_labels = []
month = -1

for index, row in district_df.iterrows():
    # create labels for hour of day
    hour_of_day = row['pub_millis'].hour
    
    if hour_of_day >= 4 and hour_of_day < 6:
        time_of_day_labels.append("em")
        
    elif hour_of_day >= 6 and hour_of_day < 9:
        time_of_day_labels.append("mr")
        
    elif hour_of_day >= 9 and hour_of_day < 12:
        time_of_day_labels.append("m")
        
    elif hour_of_day >= 12 and hour_of_day < 15:
        time_of_day_labels.append("a")
        
    elif hour_of_day >= 15 and hour_of_day < 19:
        time_of_day_labels.append("ar")
        
    elif hour_of_day >= 19 and hour_of_day < 22:
        time_of_day_labels.append("n")
        
    else: # the time is between 22 - 24 or 1 - 3
        time_of_day_labels.append("nn")
    
    
    # create labels for day of the week
    day_of_week = row['pub_millis'].dayofweek
    
    if day_of_week is 0:
        day_of_week_labels.append("mon")
        
    elif day_of_week >= 1 and day_of_week <= 3:
        day_of_week_labels.append("tues_wed_thur")
        
    elif day_of_week is 4:
        day_of_week_labels.append("fri")
        
    elif day_of_week is 5:
        day_of_week_labels.append("sat")
        
    elif day_of_week is 6:
        day_of_week_labels.append("sun")
        
    # create labels for season
    month = row['pub_millis'].month
    
    if month >= 3 and month <= 5:
        season_labels.append("spring")
        
    elif month >= 6 and month <= 8:
        season_labels.append("summer")
        
    elif month >= 9 and month <= 11:
        season_labels.append("fall")
    
    else: # the month is 12, 1, or 2
        season_labels.append("winter")


In [ ]:
# one-hot encode the features to create new columns
time_of_day_encoded = pd.get_dummies(pd.Series(time_of_day_labels))
day_of_week_encoded = pd.get_dummies(pd.Series(day_of_week_labels))
season_encoded = pd.get_dummies(pd.Series(season_labels))

In [ ]:
# reset the indexes to prevent concatenation errors
district_df.reset_index(drop=True, inplace=True)
time_of_day_encoded.reset_index(drop=True, inplace=True)
day_of_week_encoded.reset_index(drop=True, inplace=True)
season_encoded.reset_index(drop=True, inplace=True)

district_df = pd.concat([district_df, time_of_day_encoded, day_of_week_encoded, season_encoded], axis=1)
district_df

In [ ]:
district_df = district_df.drop('winter', 1)
district_df.head()

In [ ]:
district_df = pd.concat([district_df, pd.get_dummies(district_df['edge_id'])], axis=1)
district_df.head()

In [ ]:
district_df = district_df.drop('edge_id', 1)

## Train KNN to perform data imputation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
district_df.head()

In [ ]:
train_df = district_df[district_df['avg_speed'] != '']
train_df.head()

In [ ]:
test_df = district_df[district_df['avg_speed'] == '']
test_df.head()

In [ ]:
X_train = train_df.drop(columns=['pub_millis', 'avg_speed'])
y_train = train_df['avg_speed'].values.astype(float)

X_test = test_df.drop(columns=['pub_millis', 'avg_speed'])
# y_test

In [ ]:
X_train.head()

In [ ]:
y_train

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Create KNN classifier
knn = KNeighborsRegressor(n_neighbors = 3)

# Fit the classifier to the data
knn.fit(X_train,y_train)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
# append predicted values to fill in missing speed values
test_df['avg_speed'] = y_pred
test_df.head()

In [ ]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
filled_df = pd.concat([train_df, test_df], axis=0)
filled_df.head()

In [ ]:
save_df = filled_df.copy()

In [ ]:
# drop columns we aren't using anymore
save_df = save_df.drop(save_df.columns[2:14], axis=1)

In [ ]:
save_df.head()

In [ ]:
reverse_encode = save_df[save_df==1].stack().reset_index().drop(0,1)

In [ ]:
reverse_encode = reverse_encode.drop(reverse_encode.columns[0], axis=1)
reverse_encode

In [ ]:
save_df.reset_index(drop=True, inplace=True)
reverse_encode.reset_index(drop=True, inplace=True)

In [ ]:
imputed_result_df = pd.concat([save_df, reverse_encode], axis=1)

In [ ]:
imputed_result_df.head()

In [ ]:
imputed_result_df = imputed_result_df.drop(imputed_result_df.columns[2:113], axis=1)

In [ ]:
imputed_result_df = imputed_result_df.rename(columns={'level_1': 'edge_id'})
imputed_result_df.columns

In [ ]:
imputed_result_df.head()

In [ ]:
imputed_result_df = imputed_result_df.sort_values(by=['edge_id', 'pub_millis']).reset_index(drop=True)
imputed_result_df.head()

In [ ]:
# fix order of columns to match expected file structure
imputed_result_df = imputed_result_df[['pub_millis','edge_id','avg_speed']]
imputed_result_df.head()

In [ ]:
# store results in file
imputed_result_df.to_csv(f'data/financial_district_{interval}_knn.csv', index=False, header=False)